In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Reshape, Conv2D, LeakyReLU, Flatten, Conv2DTranspose, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal
import numpy as np

In [2]:
# Initialize random normal weights
initializer = RandomNormal(mean=0.0, stddev=0.02)

# Mapping Network for StyleGAN2
def mapping_network(z_dim=512, n_layers=8):
    z_input = Input(shape=(z_dim,))
    x = z_input
    for _ in range(n_layers):
        x = Dense(z_dim, activation='relu', kernel_initializer=initializer)(x)
    return Model(z_input, x)

In [3]:
# Synthesis Block (part of the Generator)
def synthesis_block(w, filters, size):
    x = Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding='same', kernel_initializer=initializer)(w)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(filters, (3, 3), padding='same', kernel_initializer=initializer)(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

In [4]:
# Synthesis Network for StyleGAN2 Generator
def synthesis_network(w_input, img_size=64):
    x = Dense(8 * 8 * 512, activation='relu', kernel_initializer=initializer)(w_input)
    x = Reshape((8, 8, 512))(x)

    # Up-sample to generate image
    x = synthesis_block(x, 256, size=16)
    x = synthesis_block(x, 128, size=32)
    x = synthesis_block(x, 64, size=64)

    x = Conv2D(3, (3, 3), padding='same', activation='tanh', kernel_initializer=initializer)(x)  # Final RGB image output
    return x

In [5]:
# Generator Model (StyleGAN2)
def build_generator(z_dim=512):
    z_input = Input(shape=(z_dim,))
    w = mapping_network(z_dim=z_dim)(z_input)
    img_output = synthesis_network(w)
    return Model(z_input, img_output)

In [6]:
# Discriminator for StyleGAN2
def build_discriminator(img_size=64):
    img_input = Input(shape=(img_size, img_size, 3))

    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer=initializer)(img_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=initializer)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer=initializer)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(512, (3, 3), strides=(2, 2), padding='same', kernel_initializer=initializer)(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    x = Dense(1, activation='sigmoid', kernel_initializer=initializer)(x)
    return Model(img_input, x)

In [7]:
# StyleGAN2 model
def build_stylegan2(z_dim=512, img_size=64):
    generator = build_generator(z_dim=z_dim)
    discriminator = build_discriminator(img_size=img_size)
    return generator, discriminator

In [10]:
!unzip /content/archive.zip -d /content/image_dataset

Archive:  /content/archive.zip
  inflating: /content/image_dataset/1/2300.png  
  inflating: /content/image_dataset/1/2301.png  
  inflating: /content/image_dataset/1/2302.png  
  inflating: /content/image_dataset/1/2303.png  
  inflating: /content/image_dataset/1/2304.png  
  inflating: /content/image_dataset/1/2305.png  
  inflating: /content/image_dataset/1/2306.png  
  inflating: /content/image_dataset/1/2307.png  
  inflating: /content/image_dataset/1/2308.png  
  inflating: /content/image_dataset/1/2309.png  
  inflating: /content/image_dataset/1/2310.png  
  inflating: /content/image_dataset/1/2311.png  
  inflating: /content/image_dataset/1/2312.png  
  inflating: /content/image_dataset/1/2313.png  
  inflating: /content/image_dataset/1/2314.png  
  inflating: /content/image_dataset/1/2315.png  
  inflating: /content/image_dataset/1/2316.png  
  inflating: /content/image_dataset/1/2317.png  
  inflating: /content/image_dataset/1/2318.png  
  inflating: /content/image_dataset/1/

In [11]:
# Loading and Preparing Anime Face Images Dataset using Keras Image Data Generator
img_width, img_height = 256, 256
batchsize = 32

train = keras. utils.image_dataset_from_directory(
    directory='/content/image_dataset',
    batch_size = batchsize,
    image_size = (img_width, img_height))

Found 3064 files belonging to 3 classes.


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Generating Augmented Batches of Anime Face Images using ImageDataGenerator
DIR = '/content/image_dataset' #path

# Create an ImageDataGenerator object with data augmentation options for image preprocessing
train_datagen = ImageDataGenerator(rescale=1./127.5,
                                   horizontal_flip = True)

train_generator = train_datagen.flow_from_directory(
        DIR,
        target_size = (64, 64),
        batch_size = batchsize,
        class_mode = None)


Found 3064 images belonging to 3 classes.


In [14]:
# Build and compile the models
generator, discriminator = build_stylegan2()

# Summarize models
generator.summary()
discriminator.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional (Functional)              │ (None, 512)                 │       2,101,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32768)               │      16,809,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 8, 8, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 16, 16, 256)         │       1,179,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 32, 32, 128)         │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 64, 64, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 3)           │           1,731 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,236,291 (81.01 MB)

 Trainable params: 21,236,291 (81.01 MB)

...

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_6 (LeakyReLU)            │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_7 (LeakyReLU)            │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_8 (LeakyReLU)            │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_9 (LeakyReLU)            │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │           8,193 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,559,169 (5.95 MB)

 Trainable params: 1,559,169 (5.95 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Compile the StyleGAN2 model
generator_optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.999)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.0004, beta_1=0.5, beta_2=0.999)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

# Training function (simplified version)
def train_stylegan(generator, discriminator, dataset, epochs=100, z_dim=512):
    for epoch in range(epochs):
        for real_images in dataset:
            # Discriminator Training
            noise = tf.random.normal([real_images.shape[0], z_dim])
            fake_images = generator(noise)
            real_labels = tf.ones((real_images.shape[0], 1))
            fake_labels = tf.zeros((real_images.shape[0], 1))

            with tf.GradientTape() as disc_tape:
                real_preds = discriminator(real_images)
                fake_preds = discriminator(fake_images)
                real_loss = loss_fn(real_labels, real_preds)
                fake_loss = loss_fn(fake_labels, fake_preds)
                d_loss = (real_loss + fake_loss) / 2

            # Update Discriminator
            gradients = disc_tape.gradient(d_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(gradients, discriminator.trainable_variables))

            # Generator Training
            noise = tf.random.normal([real_images.shape[0], z_dim])
            misleading_labels = tf.ones((real_images.shape[0], 1))  # Generator wants to fool the discriminator

            with tf.GradientTape() as gen_tape:
                fake_images = generator(noise)
                fake_preds = discriminator(fake_images)
                g_loss = loss_fn(misleading_labels, fake_preds)

            # Update Generator
            gradients = gen_tape.gradient(g_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(gradients, generator.trainable_variables))

        print(f"Epoch {epoch+1}/{epochs}, Discriminator Loss: {d_loss:.4f}, Generator Loss: {g_loss:.4f}")


In [16]:
# Build StyleGAN2 model
generator, discriminator = build_stylegan2()

# Train the model
train_stylegan(generator, discriminator,train_generator, epochs=5, z_dim=512)

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


KeyboardInterrupt: 